# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [6]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [7]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [8]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [9]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [10]:
# Getting the data from the top 10 expensive neighbourhoods
# YOUR CODE HERE!
top_10_by_price = to_data.reset_index().drop(columns="year")
top_10_by_price = top_10_by_price.groupby("neighbourhood").mean(
                ).sort_values("average_house_value", ascending=False)
top_10_by_price = top_10_by_price.iloc[:10].reset_index()

# Calculate the mean number of dwelling types units per year
# YOUR CODE HERE!
dwelling_types_by_year = to_data.groupby("year").sum(
                        ).drop(columns=["average_house_value", 
                                        "shelter_costs_owned", 
                                        "shelter_costs_rented"
                                       ]
                        )
 
# Calculate the average monthly shelter costs for owned and rented dwellings
# YOUR CODE HERE!
average_shelter_costs_by_year = to_data[["shelter_costs_owned", "shelter_costs_rented"]
                                       ].groupby("year").mean()


### Panel Visualization Functions

In [11]:
# Define Panel visualization functions
def neighbourhood_map():
    """Neighbourhood Map"""
    to_data_raw = to_data.reset_index().drop(columns="year")
    to_data_raw = to_data_raw.groupby("neighbourhood").mean(
                ).reset_index()
    
    avg_value_by_location =df_neighbourhood_locations.merge(to_data_raw, how="left")
    
    data_to_plot = avg_value_by_location[["neighbourhood", "lat", "lon", "average_house_value"]]
    
    map_plot = px.scatter_mapbox(
        data_to_plot,
        lat="lat",
        lon="lon",
        color="average_house_value",
        size="average_house_value",
        zoom=9,
        color_continuous_scale="sunset",
        title="Average House Values in Toronto"
    )
    return map_plot

def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    
    bar_chart = data.hvplot.bar(rot=90, 
                                height=500, 
                                width=400,
                                title=title, 
                                xlabel=xlabel, 
                                ylabel=ylabel, 
                                color=color,
                                grid=True,
                                ylim=(0,500000)
                               ).opts(yformatter="%.0f")
    return bar_chart


def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    
    line_chart = data.hvplot.line(rot=90, 
                                  height=400, 
                                  width=600,
                                  title=title, 
                                  xlabel=xlabel, 
                                  ylabel=ylabel, 
                                  color=color,
                                  grid=True
                               ).opts(yformatter="%.0f")
    return line_chart

def average_house_value():
    """Average house values per year."""
    
    average_house_value = to_data["average_house_value"].groupby("year").mean()
    
    avg_house_value = average_house_value.hvplot.line(title="Average House Value in Toronto",
                              grid=True,
                              xlabel="Year",
                              ylabel="Avg House Value ($)"
                             ).opts(yformatter="%.0f")
    return avg_house_value

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    
    avg_value_by_nbhd = to_data[["neighbourhood", "average_house_value"]].reset_index()
    avg_value_by_nbhd.rename(columns={"neighbourhood":"Neighbourhood"}, inplace=True)

    average_value_by_nbhd = avg_value_by_nbhd.hvplot.line(x="year", 
                              xlabel="Year", 
                              ylabel="Avg House Value", 
                              groupby="Neighbourhood", 
                              c="plum",
                              grid=True,
                              ylim=(0,2200000)
                             ).opts(yformatter="%.0f")
    return average_value_by_nbhd

def number_dwelling_types():
    """Number of dwelling types per year"""
    
    dwelling_types_by_nbhd = to_data.drop(columns=["average_house_value", 
                                        "shelter_costs_owned", 
                                        "shelter_costs_rented"
                                       ]
                                     ).reset_index()
    dwelling_types_by_nbhd.rename(columns={"neighbourhood":"Neighbourhood"}, inplace=True)
    
    dwelling_types_by_neighbourhood = dwelling_types_by_nbhd.hvplot.bar(rot=90, 
                                  x="year", 
                                  xlabel="Year", 
                                  height=500, 
                                  ylabel="Dwelling Type Units", 
                                  groupby="Neighbourhood",
                                  grid=True,
                                  ylim=(0,8000)
                                )
    return dwelling_types_by_neighbourhood

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""

    faceted_row_bar_chart = px.bar(to_data.reset_index(),
        x="neighbourhood",
        y="average_house_value",
        facet_row="year",
        color="average_house_value",
        labels=({"neighbourhood":"Neighbourhood", "average_house_value":"Avg. House Value"}),
        height=900,
        title="Average House Values in Toronto per Neighbourhood"
      )
    return faceted_row_bar_chart

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    
    top_10_by_price = to_data.reset_index().drop(columns="year")
    top_10_by_price = top_10_by_price.groupby("neighbourhood").mean(
                ).sort_values("average_house_value", ascending=False)
    top_10_by_price = top_10_by_price.iloc[:10].reset_index()
    
    top_10_nbhds = top_10_by_price.hvplot.bar(rot=90,
                           x="neighbourhood", 
                           xlabel="Neighbourhood", 
                           height=500, 
                           y="average_house_value", 
                           ylabel="Avg House Value ($)", c="silver",
                           title="Top 10 Most Expensive Neighbourhoods in Toronto",
                           grid=True
                          ).opts(yformatter="%.0f")
    return top_10_nbhds

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    
    sunburst = px.sunburst(to_data.reset_index(),
            path=["year", "neighbourhood"],
            values="average_house_value",
            color_continuous_scale="rdbu",
            color="average_house_value",
            title="Cost Analysis of Most Expensive Neighbourhoods in Toronto per Year"
           )
    return sunburst


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [12]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
dashboard_title = "# Real Estate Analysis of Toronto (2001-2016)"
# Define a welcome text
# YOUR CODE HERE!
welcome_text = "This dashboard analyses and presents Toronto dwelling types and average house values for the years 2001, 2006, 2011 and 2016"
# Create a tab layout for the dashboard
# YOUR CODE HERE!
welcome_column = pn.Column(
    welcome_text,
    neighbourhood_map()
)
yearly_market_analysis_row1 = pn.Row(
    create_bar_chart(dwelling_types_by_year.iloc[0], 
                     "Distribution of Dwelling Types (2001)", 
                     "Dwelling Types", 
                     "Count", 
                     "pink"
                    ),
    create_bar_chart(dwelling_types_by_year.iloc[1], 
                     "Distribution of Dwelling Types (2006)",
                     "Dwelling Types", 
                     "Count", 
                     "gold"
                    )
)
yearly_market_analysis_row2 = pn.Row(
    create_bar_chart(dwelling_types_by_year.iloc[2], 
                     "Distribution of Dwelling Types (2011)", 
                     "Dwelling Types", 
                     "Count", 
                     "teal"
                    ),
    create_bar_chart(dwelling_types_by_year.iloc[3], 
                     "Distribution of Dwelling Types (2016)", 
                     "Dwelling Types", 
                     "Count", 
                     "lavender"
                    )
)
yearly_market_analysis_col= pn.Column(
    yearly_market_analysis_row1,
    yearly_market_analysis_row2
)
shelter_cost_col = pn.Column(
    average_house_value(),
    create_line_chart(average_shelter_costs_by_year["shelter_costs_owned"], 
                      "Average Monthly Shelter Cost for Owned Dwellings in Torronto",  
                      "Year", 
                      "Avg Monthly Shelter Costs ($)",
                      "green"
                     ),
    create_line_chart(average_shelter_costs_by_year["shelter_costs_rented"], 
                      "Average Monthly Shelter Cost for Rented Dwellings in Torronto",  
                      "Year", 
                      "Avg Monthly Shelter Costs ($)", 
                      "purple"
                     )
)
nbhd_analysis_col = pn.Column(
    number_dwelling_types(),
    average_value_by_neighbourhood()
)
nbhd_analysis_row = pn.Row(
    nbhd_analysis_col,
    average_house_value_snapshot()
)
top_expensive_row = pn.Row(
    top_most_expensive_neighbourhoods(),
    sunburts_cost_analysis()
)
# Create the main dashboard
# YOUR CODE HERE!
dashboard = pn.Tabs(
    ("Welcome", welcome_column),
    ("Yearly Market Analysis", yearly_market_analysis_col),
    ("Shelter Costs Vs. Houses Value", shelter_cost_col),
    ("Neighbourhood Analysis", nbhd_analysis_row),
    ("Top Expensive Neighbourhoods", top_expensive_row)
)

## Serve the Panel Dashboard

In [58]:
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
    [2] Column
        [0] HoloViews(Curve)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [3] Row
        [0] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='Neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                    [1] VSpacer()
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='Neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                    [1] VSpacer()
        [1] Plotly(Figure)
    [4] Row
        [0] HoloViews(Bars)
        [1] Plotly(Figure)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()